# Scraping web data and text mining for analysis, and for use with a Tensorflow RNN example

In [14]:
import os
os.getcwd()

'/Users/MF/Desktop/data-science-side-projects/data_scraping_project'

In [5]:
#from HTMLParser import HTMLParser
#parser = HTMLParser()
#parser.feed('https://docs.python.org/2/library/htmlparser.html')

In [10]:
#%lsmagic
#!ps -a
##%time
##2+2

Example using scrapy module for data scraping, see: https://doc.scrapy.org/en/1.4/intro/overview.html

In [17]:
with open('quotes_spider.py', 'w') as swf:
    swf.write('''\
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/tag/humor/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.xpath('span/small/text()').extract_first(),
            }

        next_page = response.css('li.next a::attr("href")').extract_first()
        if next_page is not None:
            yield response.follow(next_page, self.parse)
            ''')
    swf.close()

In [20]:
! scrapy runspider quotes_spider.py -o quotes.json &>/dev/null

In [1]:
#pip install beautifulsoup4
#pip install urllib
#also consider lxml package instead of bs4?
#also a package called bleach (from mozilla?)
from bs4 import BeautifulSoup
import urllib2

webpage = urllib2.urlopen('http://en.wikipedia.org/wiki/Main_Page')
soup = BeautifulSoup(webpage,'html.parser')
for anchor in soup.find_all('a'):
    print(anchor.get('href', '/'))

/
#mw-head
#p-search
/wiki/Wikipedia
/wiki/Free_content
/wiki/Encyclopedia
/wiki/Wikipedia:Introduction
/wiki/Special:Statistics
/wiki/English_language
/wiki/Portal:Arts
/wiki/Portal:Biography
/wiki/Portal:Geography
/wiki/Portal:History
/wiki/Portal:Mathematics
/wiki/Portal:Science
/wiki/Portal:Society
/wiki/Portal:Technology
/wiki/Portal:Contents/Portals
/wiki/Tidus
/wiki/Role-playing_video_game
/wiki/Protagonist
/wiki/Final_Fantasy_X
/wiki/Blitzball_(Final_Fantasy)
/wiki/Final_Fantasy_X-2
/wiki/Kingdom_Hearts
/wiki/Square_Enix
/wiki/Tetsuya_Nomura
/wiki/Kazushige_Nojima
/wiki/James_Arnold_Taylor
/wiki/Masakazu_Morita
/wiki/Motion_capture
/wiki/Character_arc
/wiki/Cosplay
/wiki/Tidus
/wiki/Ram_Narayan
/wiki/Melbourne_Castle
/wiki/Auriscalpium_vulgare
/wiki/Wikipedia:Today%27s_featured_article/June_2017
https://lists.wikimedia.org/mailman/listinfo/daily-article-l
/wiki/Wikipedia:Featured_articles
/wiki/File:Agagianian_1958_(crop).jpg
/wiki/Gregorio_Pietro_Agagianian
/wiki/Washington_St

In [10]:
help(anchor.find_all)

Help on method find_all in module bs4.element:

find_all(self, name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs) method of bs4.element.Tag instance
    Extracts a list of Tag objects that match the given
    criteria.  You can specify the name of the Tag and any
    attributes you want the Tag to have.
    
    The value of a key-value pair in the 'attrs' map can be a
    string, a list of strings, a regular expression object, or a
    callable that takes a string and returns whether or not the
    string matches for some custom definition of 'matches'. The
    same is true of the tag name.



http://adequateman.deadspin.com/whats-the-best-store-to-daydream-about-robbing-1796232076
http://deadspin.com/tag/funbag

IDEA 1: maybe have the scrapyspider collect the urls, and then run through beautiful soup like below and write each one to its own text file. Perhaps this is formally two processes, web crawling with spider, and then html parsing and cleanup for each page

there's also some type of querying language with an x in it to use...

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

IDEA 2: For the projects... one version reproduces whole column. another version you enter a question and it generates an answer for you... would be interesting to do for an "agony aunt" column... should be a lot of data to scrape there...

TO DO: 
- Fix up formatting so apostrophes don't get messed up in html (.prettify(formatter="html")) !!!!!
- Figure out how to not repeat the contents of blockquote
- Crawl the site to make a file of all the urls. Print that to text file, so it can be loaded in and looped into bs4 stuff below

In [43]:
import os
from bs4 import BeautifulSoup
import urllib2

my_url = 'http://adequateman.deadspin.com/whats-the-best-store-to-daydream-about-robbing-1796232076'
my_outfile = 'first.html'

def oneFullPageParse(url, outfile):
    fo = open(outfile, 'w')
    funbag = urllib2.urlopen(url)
    soup = BeautifulSoup(funbag,'html5lib') #html.parser')
    for anchor in soup.find_all(['p', 'blockquote'], class_ = ''): 
        if anchor.parent.name == 'blockquote':
            continue
        fo.write(anchor.prettify(formatter="html")) #anchor.prettify() #anchor.contents
    fo.close()
    
oneFullPageParse(my_url, my_outfile)


In [37]:
    
    #'p', class_ = '' #removes the p tags that have a class attribute!!! just need blockquotes
    #['p', 'blockquote'] # almost right, but repeats the contents of the blockquote, maybe can do something with parent/child
    #'p' and 'blockquote' #gets the two used together... could just generate questions with this for a start
    #'p' or 'blockquote' #doesn't print out the blockquote tag but has the data...
    #print anchor.parent #do something with this, it works! find parent though?!
    
    #print anchor.previousSibling
    ###print soup.find_parent('anchor')
    
    #maybe have something like if parent of p is block quote, do nothing <<<<<<<<<<<<
    
#looks like we want to strip out certain classes for p, like the ad label. maybe do with text processing outside bs4
#do that, and throw in the blockquotes, and bob's your uncle?

In [24]:
#print(soup.prettify())

In [34]:
mytest = BeautifulSoup('<a><blockquote><p>Text</p></blockquote></a>')

print mytest
print mytest.blockquote
print mytest.find('blockquote')
myp = mytest.find_all('p') #works for find withou subscripting, need [0] for find_all
print myp[0]

print myp[0].parent.name == 'blockquote'
print myp[0].parent.find('blockquote')

print tag.find

#print myp[0].find_parent()

#almost = myp[0].find_parent()
#print almost

#print almost.find_all('blockquote')

##if (myp[0].find_parent()).find('blockquote'):
##    print 'got it!!!'

#mytag = soup.p
#print mytag
#print soup.find_parent('p')


<html><body><a><blockquote><p>Text</p></blockquote></a></body></html>
<blockquote><p>Text</p></blockquote>
<blockquote><p>Text</p></blockquote>
<p>Text</p>
True
None


NameError: name 'tag' is not defined

In [18]:
print mytag.string

None


In [38]:
css_soup = BeautifulSoup('<p class="body"></p>')
print css_soup.p['class']
print css_soup.p.get_attribute_list('class')

['body']
['body']


In [49]:
print soup.blockquote.contents

[<p>If you could rob one store overnight with a zero percent chance of being caught, which store would you pick? Best Buy immediately jumps to mind (give me all them big ass TVs) but something like Macy\u2019s or a Costco would probably make more sense for life-living purposes, right?</p>]


In [42]:
import os
os.getcwd()+'/stuff'

'/Users/MF/Desktop/data-science-side-projects/data_scraping_project/stuff'